In [1]:
## Set up SAM

In [5]:
import torch
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [2]:
from transformers import SamProcessor
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

2024-04-11 12:27:13.414996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 12:27:14.212164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from transformers import SamModel
model = SamModel.from_pretrained("facebook/sam-vit-base")

/home/jeru/miniconda3/envs/llama-2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## Data Preprocessing

In [ ]:
import torchvision.transforms as transforms

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),                              # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5]),       # Normalize pixel values to range [-1, 1]
    transforms.RandomAffine(degrees=0, shear=0.2),      # Shear
    transforms.RandomAffine(degrees=0, scale=(1-0.2, 1+0.2)),   # Zoom
    transforms.RandomHorizontalFlip(p=0.5)             # Horizontal flip with probability 0.5
])


dataset = torchvision.datasets.ImageFolder(root='path/to/dataset', transform=transform)


In [ ]:
##Define DeepPhase Model Structure

In [11]:
import torch.nn as nn
import torch.nn.functional as F

# Update the CNN architecture for a 3-class classification
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        # Fully connected layer for 3 classes
        self.fc = nn.Linear(17*17*32, 3)
        
    def forward(self, x):
        # Apply convolutions and max pooling layers
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv3(x)), kernel_size=2)
        # Flatten the tensor
        x = x.view(-1, 17*17*32)
        # Fully connected layer with no activation, activation will be included in the loss function (e.g., CrossEntropyLoss)
        x = self.fc(x)
        return x

# Instantiate the updated model for 3-class classification
model = SimpleCNN()


In [12]:
import torch
import torch.nn as nn

# Define the custom loss function
class CustomLoss(nn.Module):
    def __init__(self, beta):
        super(CustomLoss, self).__init__()
        self.beta = beta

    def forward(self, predictions, targets):
        # Mean Squared Error
        mse = torch.mean((targets - predictions) ** 2)
        # Mean Absolute Error
        mean_difference = torch.mean(torch.abs(targets - predictions))
        # Custom loss as per the provided equation
        loss = mse + self.beta * mean_difference
        return loss

# Example usage of the custom loss function
# Assuming beta is provided
beta_value = 0.5  # This can be any non-negative value, provided as hyperparameter
custom_loss_function = CustomLoss(beta=beta_value)

# Example tensors for predictions and targets
predictions_example = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
targets_example = torch.tensor([0.0, 0.25, 0.5])

# Calculate loss
loss = custom_loss_function(predictions_example, targets_example)
loss.backward()  # Backpropagate to compute gradients


In [ ]:
## Model Training Loop

In [ ]:
# Prepare dataset
train_dataset = MyDataset()
val_dataset = MyDataset()
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from sklearn.model_selection import ParameterGrid

# Assuming SimpleCNN, CustomLoss, and MyDataset are already defined

# Define the parameter grid for hyperparameter search
param_grid = {
    'lr': [0.00001],
    'batch_size': [10]
}

# Define the number of epochs
num_epochs = 50

# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            # Get the inputs and labels
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            # Compute the loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

# Define the evaluation loop
def evaluate_model(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(val_loader)

# Prepare the datasets and dataloaders outside of the grid search loop to save time
train_dataset = MyDataset()  # You need to implement this
val_dataset = MyDataset()    # You need to implement this

# Conduct the grid search
best_params = None
best_val_loss = float('inf')

for params in ParameterGrid(param_grid):
    model = SimpleCNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=params['lr'])
    criterion = CustomLoss(beta=0.2).to(device)
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
    
    # Train the model for a fixed number of epochs
    train_model(model, train_loader, criterion, optimizer, num_epochs)
    
    # Evaluate the model
    val_loss = evaluate_model(model, val_loader, criterion)
    
    # Update the best parameters if the current model is better
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_params = params
        # Optionally, save the best model
        torch.save(model.state_dict(), 'best_model.pth')

# Output the best parameters found
print(f"Best parameters found: {best_params}")

# Optionally, load the best model for further use or evaluation
model_to_use = SimpleCNN()
model_to_use.load_state_dict(torch.load('best_model.pth'))
model_to_use.to(device)
model_to_use.eval()
